In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install geocoder
!pip install folium
!pip install geopy

import requests
import pandas as pd
import geocoder
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np
import geopy

from bs4 import BeautifulSoup as bsoup
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print('Packages fully loaded and installed.')

Packages fully loaded and installed.


In [16]:
#define URL source
source = requests.get('https://en.wikipedia.org/wiki/Neighborhoods_in_Boston').text

#create a Beautiful Soup object and define lxml as the parser
soup= bsoup(source, 'lxml')
divlist = soup.find('div',class_ = 'div-col')
ulist = divlist.ul
ulist = ulist.text.split('\n')
ulist

['Allston',
 'Back Bay',
 'Bay Village',
 'Beacon Hill',
 'Brighton',
 'Charlestown',
 'Chinatown/Leather District',
 'Dorchester (divided for planning purposes into Mid Dorchester and Dorchester)',
 'Downtown',
 'East Boston',
 'Fenway Kenmore (includes Longwood)',
 'Hyde Park',
 'Jamaica Plain',
 'Mattapan',
 'Mission Hill',
 'North End',
 'Roslindale',
 'Roxbury',
 'South Boston',
 'South End',
 'West End',
 'West Roxbury']

In [37]:
df=pd.DataFrame(ulist)
df.columns = ['Neighborhood']
df.replace(['Dorchester (divided for planning purposes into Mid Dorchester and Dorchester)'],['Dorchester'], inplace=True)
df.replace(['Fenway Kenmore (includes Longwood)'], ['Fenway Kenmore'], inplace = True)
df.replace(['Chinatown/Leather District'],['Chinatown'], inplace=True)
df

,Neighborhood
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton
5,Charlestown
6,Chinatown
7,Dorchester
8,Downtown
9,East Boston


In [38]:
#create a function to handle TimeOuts from Geocoder
from geopy.exc import GeocoderTimedOut
locator = Nominatim(user_agent = "bostonagent")
def do_geocode(address):
    try:
        return locator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [44]:
neighborhoods = df.values.tolist()

latitude = []
longitude = []
for neighborhood in neighborhoods:
    coord = do_geocode('{}, Boston, Massachusetts'.format(neighborhood))
    #check to make sure all latitude and longitude values are present, handles the case where Nominatim returns None
    if (coord == None):
        latitude.append('0')
        longitude.append('0')
    else:
        latitude.append(coord.latitude)
        longitude.append(coord.longitude)

#add coordinates column to dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude

In [45]:
df

,Neighborhood,Latitude,Longitude
0,Allston,42.355434,-71.132127
1,Back Bay,42.350707,-71.079730
2,Bay Village,42.350011,-71.066948
3,Beacon Hill,42.358708,-71.067829
4,Brighton,42.350097,-71.156442
5,Charlestown,42.377875,-71.061996
6,Chinatown,42.351329,-71.062623
7,Dorchester,42.297320,-71.074495
8,Downtown,42.362918,-71.068737
9,East Boston,42.375097,-71.039217


In [46]:
address = 'Boston, Massachusetts'

#Get lat/long values for Boston and print them
location = locator.geocode(address)
lat = location.latitude
long = location.longitude
print('The geograpical coordinates of Boston are {}, {}.'.format(lat, long))

The geograpical coordinates of Boston are 42.3602534, -71.0582912.


In [47]:
boston_map = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map for each neighborhood
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='cadetblue',
        fill=True,
        fill_color='#29BBB9',
        fill_opacity=0.7,
        parse_html=False).add_to(boston_map)  
    
boston_map

In [48]:
CLIENT_ID = 'GPEZ2BHSQDPGBGF2NFK3QT2PZRFIJJS015OE1XMUICVQM1HK' # your Foursquare ID
CLIENT_SECRET = 'RO4RBQEB2YOIN13PUIPLFCHF4XN1YU40ZVPCMITB2Y0D1JEA' # your Foursquare Secret
VERSION = '20191227'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GPEZ2BHSQDPGBGF2NFK3QT2PZRFIJJS015OE1XMUICVQM1HK
CLIENT_SECRET:RO4RBQEB2YOIN13PUIPLFCHF4XN1YU40ZVPCMITB2Y0D1JEA


In [53]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
LIMIT = 100
radius = 500
boston_venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'])

Allston
Back Bay
Bay Village
Beacon Hill
Brighton
Charlestown
Chinatown
Dorchester
Downtown
East Boston
Fenway Kenmore
Hyde Park
Jamaica Plain
Mattapan
Mission Hill
North End
Roslindale
Roxbury
South Boston
South End
West End
West Roxbury


In [55]:
print(boston_venues.shape)
boston_venues.head()

(723, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allston,42.355434,-71.132127,Lulu's Allston,42.355068,-71.134107,Comfort Food Restaurant
1,Allston,42.355434,-71.132127,Kaju Tofu House,42.354329,-71.132374,Korean Restaurant
2,Allston,42.355434,-71.132127,Azama Grill,42.354422,-71.132358,Falafel Restaurant
3,Allston,42.355434,-71.132127,Allston Diner,42.354979,-71.134295,Diner
4,Allston,42.355434,-71.132127,BonChon Chicken,42.353105,-71.130921,Fried Chicken Joint


In [56]:
print('There are {} unique categories.'.format(len(boston_venues['Venue Category'].unique())))
boston_venues.groupby('Neighborhood').count()

There are 73 unique categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allston,73,73,73,73,73,73
Back Bay,56,56,56,56,56,56
Bay Village,47,47,47,47,47,47
Beacon Hill,32,32,32,32,32,32
Brighton,23,23,23,23,23,23
Charlestown,20,20,20,20,20,20
Chinatown,100,100,100,100,100,100
Dorchester,8,8,8,8,8,8
Downtown,28,28,28,28,28,28


In [61]:
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood names back to dataframe
boston_onehot['Neighborhood'] = boston_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1] ]+ list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()
print(boston_onehot.columns)

Index(['Neighborhood', 'American Restaurant', 'Arepa Restaurant',
       'Asian Restaurant', 'BBQ Joint', 'Bagel Shop', 'Bakery',
       'Belgian Restaurant', 'Bistro', 'Brazilian Restaurant',
       'Breakfast Spot', 'Burger Joint', 'Burmese Restaurant', 'Burrito Place',
       'Cafeteria', 'Café', 'Caribbean Restaurant', 'Chinese Restaurant',
       'Colombian Restaurant', 'Comfort Food Restaurant', 'Creperie',
       'Cuban Restaurant', 'Deli / Bodega', 'Dim Sum Restaurant', 'Diner',
       'Donut Shop', 'Dumpling Restaurant', 'Eastern European Restaurant',
       'Falafel Restaurant', 'Fast Food Restaurant', 'Food', 'Food Court',
       'Food Stand', 'Food Truck', 'French Restaurant', 'Fried Chicken Joint',
       'Gastropub', 'Greek Restaurant', 'Hot Dog Joint', 'Hotpot Restaurant',
       'Indian Restaurant', 'Irish Pub', 'Italian Restaurant',
       'Japanese Restaurant', 'Korean Restaurant', 'Latin American Restaurant',
       'Malay Restaurant', 'Mediterranean Restaurant', 'Me

In [62]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
boston_grouped

,Neighborhood,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,...,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Allston,0.000000,0.000000,0.082192,0.000000,0.000000,0.068493,0.000000,0.000000,0.027397,...,0.000000,0.013699,0.00,0.013699,0.013699,0.000000,0.041096,0.000000,0.041096,0.027397
1,Back Bay,0.125000,0.000000,0.017857,0.000000,0.017857,0.000000,0.000000,0.000000,0.017857,...,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.035714
2,Bay Village,0.063830,0.000000,0.021277,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,...,0.063830,0.042553,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000
3,Beacon Hill,0.218750,0.000000,0.000000,0.000000,0.031250,0.031250,0.000000,0.031250,0.000000,...,0.031250,0.062500,0.00,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000
4,Brighton,0.043478,0.000000,0.000000,0.000000,0.000000,0.130435,0.000000,0.000000,0.000000,...,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000
5,Charlestown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000
6,Chinatown,0.020000,0.000000,0.080000,0.000000,0.000000,0.090000,0.000000,0.010000,0.000000,...,0.020000,0.050000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000
7,Dorchester,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000
8,Downtown,0.107143,0.000000,0.000000,0.035714,0.035714,0.000000,0.000000,0.035714,0.000000,...,0.000000,0.035714,0.00,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000
9,East Boston,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250


In [65]:
num_top_venues = 10

for hood in boston_grouped['Neighborhood']:
    print("----Top Ten Venues in "+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Top Ten Venues in Allston----
                           Venue  Frequency
0              Korean Restaurant       0.10
1               Asian Restaurant       0.08
2             Chinese Restaurant       0.07
3                         Bakery       0.07
4             Italian Restaurant       0.05
5                    Pizza Place       0.05
6  Vegetarian / Vegan Restaurant       0.04
7                Thai Restaurant       0.04
8                           Food       0.04
9          Vietnamese Restaurant       0.03


----Top Ten Venues in Back Bay----
                      Venue  Frequency
0       American Restaurant       0.12
1        Seafood Restaurant       0.11
2        Italian Restaurant       0.11
3   New American Restaurant       0.07
4            Sandwich Place       0.05
5        Mexican Restaurant       0.05
6             Deli / Bodega       0.04
7                Steakhouse       0.04
8                Restaurant       0.04
9  Mediterranean Restaurant       0.04


----Top Ten Ve